# Calculator Bool Exam

|Entrada| Salida |
|:-:|:-:|
|x := 2;||
|print x;|Resultado es verdadero|
|y := 0 and x; | |
|print nor y;| Resultado es verdadero|
|print x and not y;| Resultado es verdader|
|print not (x and not y) ; | Resultado es falso |
| x:= not x; | |
|z:= true or not (x and not y); ||
|w := (3 > 5)  and (z < 4);||
|print 4 == (w < 3);| Resultado es falso |



## Grammar and Traduction scheme

```
input -> `print` expr {escribe(expr.s)};
input -> asig;

asig -> ID `=` expr {table[ID.lexval] = expr.s}

expr -> expr_1 `or` exprOR { espr.s = expr_1.s || exprOR.s}
expr -> exprOR {expr.s = exprOR.s}

exprOR -> exprOR_1 `and` exprAND { exprOR.s = exprOR_1.s && exprAND.s}
exprOR -> exprAND {exprOR.s = exprAND.s}

exprAND -> boolean {exprAND.s = boolean.s}

boolean -> not boolean_1 {boolean.s = !boolean_1.s}
boolean -> CBOOLEAN {boolean.s = CBOOLEAN}
boolean -> ID {boolean.s = table[ID.Lexval]}
boolean -> (expr) {boolean.s = expr.s}
```

## L-recursive to R-recursive

```
input -> `print` expr {escribe(expr.s)};
input -> asig;
asig -> ID `=` expr {table[ID.lexval] = expr.s}

expr -> exprOR {expr'.h = exprOR.s} expr' {expr.s = expr'.s}
expr' -> `||` exprOR {expr'_1.h = expr'.h or exprOR.s}  expr'_1 {expr'.s = expr'_1.s}
expr' -> e {expr'.s = expr'.h}

exprOR -> exprAND {exprOR'.h = exprAND.s} exprOR' {exprOR.s = exprOR'.s}
exprOR' -> `and` exprAND {exprOR'_1.h = exprOR'.h and exprAND.s} expOR'_1 {exprOR'.s = exprOR'_1.s}
exprOR' -> e {exprOR'.s = exprOR'.h}
exprAnd -> boolean {exprOR'.s = boolean.s}

boolean -> not boolean_1 {boolean.s = !bolean_1.s}
boolean -> CBOOL {boolean.s = CBOOL.lexval}
boolean -> ID {boolean.s = table[ID.lexval]}
boolean -> `(`expr`)` {boolean.s = expr.s}

```

## First and Follow Set

### First set

|First | Elementos|
|:-:|:-:|
|boolean| {CBOOL, 'ID', '(', 'not'} |
|exprAND| {CBOOL, 'ID', '(', 'not'} |
|exprOR | {CBOOL, 'ID', '(', 'not'} |
|exprOR' | { 'and' , e, } |
|expr' | {`or`, e} | 
|expr | {CBOOL, 'ID', '(', 'not'} |
| asig | {'ID'} |
| input | {'ID', 'print'} |


### Follow set

|Follow| Elementos|
|:-:|:-:|
|input |{$}|
|asig|';' '\)'|
|expr|';' '\)'|
|expr'|';' '\)' |
|exprOR|'or' ';' '\)' |
|exprOR'| 'or' ';' '\)' |
|exprAND| 'and', 'or' ';' '\)' |
|boolean | 'and' 'or' ';' '\)' |

## Translation scheme AST

```
input -> `print` expr {expr.s.escribe()};
input -> asig;
asig -> ID `=` expr {BinaryOpNode(=, NodeID(ID.lexval), expr.s)}

expr -> exprOR {expr'.h = exprOR.s} expr' {expr.s = expr'.s}
expr' -> `||` exprOR {expr'_1.h = BinaryOpNode('or', expr'.h, exprOR.s)}  expr'_1 {expr'.s = expr'_1.s}
expr' -> e {expr'.s = expr'.h}

exprOR -> exprAND {exprOR'.h = exprAND.s} exprOR' {exprOR.s = exprOR'.s}
exprOR' -> `and` exprAND {exprOR'_1.h = BinaryOpNode('and', exprOR'.h, exprAND.s)} expOR'_1 {exprOR'.s = exprOR'_1.s}
exprOR' -> e {exprOR'.s = exprOR'.h}
exprAnd -> boolean {exprOR'.s = boolean.s}

boolean -> not boolean_1 {boolean.s = UnaryOpNode('!', bolean_1.s)}
boolean -> CBOOL {boolean.s = NodeBool(CBOOL.lexval)}
boolean -> ID {boolean.s = NodeID(ID.lexval)}
boolean -> `(`expr`)` {boolean.s = expr.s}

```

## Lexer


In [17]:
from sly import Lexer

class BoolLexer(Lexer):

    ignore = ' \t'
    tokens = {
        CBOOL, ID, OR, AND, NOT, PRINT
    }

    literals = {
        '=', ';', '(', ')',
    }

    CNUM

    # regular expresions
    ID = r'[a-zA-Z_][a-zA-Z0-9]*'
    ID['false'] = CBOOL
    ID['true'] = CBOOL
    ID['OR'] = OR 
    ID['AND'] = AND
    ID['print'] = PRINT
    ID['not'] = NOT 

    # Auxiliar functions
    @_(r'\n+')
    def newline(self, t):
        self.lineno += t.value.count('\n')

    def error(self, t):
        print("line {}: Bad character {}".format(self.lineno, t.value[0]))
        self.index += 1


## Prueba Lexer


In [21]:
if __name__ == '__main__':
    data = ['z = true OR NOT (x and not y);',
            'w = (true OR  false) and (true and false);', ]

    lexer = BoolLexer()
    for sentence in data:
        for token in lexer.tokenize(sentence):
            	print(token)
        print("*" * 60)


Token(type='ID', value='z', lineno=1, index=0)
Token(type='=', value='=', lineno=1, index=2)
Token(type='CBOOL', value='true', lineno=1, index=4)
Token(type='OR', value='OR', lineno=1, index=9)
Token(type='ID', value='NOT', lineno=1, index=12)
Token(type='(', value='(', lineno=1, index=16)
Token(type='ID', value='x', lineno=1, index=17)
Token(type='ID', value='and', lineno=1, index=19)
Token(type='NOT', value='not', lineno=1, index=23)
Token(type='ID', value='y', lineno=1, index=27)
Token(type=')', value=')', lineno=1, index=28)
Token(type=';', value=';', lineno=1, index=29)
************************************************************
Token(type='ID', value='w', lineno=1, index=0)
Token(type='=', value='=', lineno=1, index=2)
Token(type='(', value='(', lineno=1, index=4)
Token(type='CBOOL', value='true', lineno=1, index=5)
Token(type='OR', value='OR', lineno=1, index=10)
Token(type='CBOOL', value='false', lineno=1, index=14)
Token(type=')', value=')', lineno=1, index=19)
Token(type='ID

## Nodes


In [ ]:

var_tab = {}
next_token = None 
index = 0 
tokens = None 
token_list = None

class Node():
    def write():
        pass

class NodeType(Node):
    def __init__(self, type) -> None:
        super().__init__()
        self.type = type

    def write(self):
        return self.type
        

class NodePointer(Node):
    def __init__(self, n):
        self.ptr = n
    
    def write(self):
        return "Pointer to {}".format(self.ptr.write())


In [4]:
entero = NodeType('entero')
print(entero.write())

entero
